# Homework 2: Word Similarity

Student Name:Junxu ZHANG

Student ID:901264

## General info

<b>Due date</b>: Thursday, 18 Apr 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 7% of mark for class (with 6% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib and Scikit-Learn. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>. 

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dateset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/">Similarity-353</a>. You need to first obtain this dataset, which is available on LMS. The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this homework, we will be treating the paragraphs of the Brown corpus as our "documents". You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized. Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of less than 8 in this corpus.

For this part, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *filtered_gold_standard*.

(1 mark)

In [149]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

###
# Your answer BEGINS HERE
###
lemmatize=nltk.stem.WordNetLemmatizer()      
f = open ('C:\\Users\\Scott\\Downloads\\set1.tab','r')
each_line=f.readlines()
for line in range(1,len(each_line)):
    line_words=each_line[line].split('\t')
    if not (line_words[0].isdigit()) and not(line_words[1].isdigit()):        
        word1, word2=line_words[0], line_words[1]
        filtered_gold_standard[(word1,word2)]=float(line_words[2].strip())
f.close()

modified_paras, less_fre_words, removed_words = [],[],[]
#print(brown.paras())  
for para in brown.paras():
    cur_para=[]
    for sentence in para:
        cur_sen=[]
        for para_token in sentence:
            if para_token.isalpha():
                lower_token=para_token.lower()
                lemmatized_token=lemmatize.lemmatize(lower_token)
                cur_sen.append(lemmatized_token)
            else:
                removed_words.append(para_token)
        cur_para.append(cur_sen)
    modified_paras.append(cur_para)
new_doc={}
for key in filtered_gold_standard.keys():
    new_doc[key[0]], new_doc[key[1]]=0, 0
for para in modified_paras:
    word_set=[]
    for sentence in para:
        for word in sentence:
            word_set.append(word)
    for word in new_doc.keys():
        if word in word_set:
            new_doc[word]+=1

for key in filtered_gold_standard.keys():
    if new_doc[key[0]] < 8 or new_doc[key[1]]<8:
        less_fre_words.append((key[0],key[1]))

for key in less_fre_words:
    del filtered_gold_standard[(key[0],key[1])]
###
# Your answer ENDS HERE
###

print(len(filtered_gold_standard))
print(filtered_gold_standard)


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Scott\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Scott\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
94
{('love', 'sex'): 6.77, ('tiger', 'cat'): 7.35, ('tiger', 'tiger'): 10.0, ('book', 'paper'): 7.46, ('plane', 'car'): 5.77, ('train', 'car'): 6.31, ('telephone', 'communication'): 7.5, ('television', 'radio'): 6.77, ('drug', 'abuse'): 6.85, ('bread', 'butter'): 6.19, ('doctor', 'nurse'): 7.0, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('smart', 'student'): 4.62, ('smart', 'stupid'): 5.81, ('company', 'stock'): 7.08, ('stock', 'market'): 8.08, ('stock', 'phone'): 1.62, ('stock', 'egg'): 1.81, ('stock', 'live'): 3.73, ('stock', 'life'): 0.92, ('book', 'library'): 7.46, ('bank', 'money'): 8.12, ('wood', 'forest'): 7.73, ('money', 'cash'): 9.08, ('king', 'queen'): 8

<b>For your testing: </b>

In [47]:
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)

In [48]:
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either a) having only one sense (i.e. only one synset), or b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria.

When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *final_gold_standard*.

(1 mark)

In [150]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}

###
# Your answer BEGINS HERE
###
word_primary_sense ={}

def filter_condition(word):
    if len(wordnet.synsets(word)) <= 1 and wordnet.synsets(word)[0].pos() == wordnet.NOUN:
        word_primary_sense[word] = wordnet.synsets(word)[0]
        return True
    total_counts = []
    for synset in wordnet.synsets(word):
        count_num = 0
        for curren_lemma in synset.lemmas():
            if curren_lemma.name() == word:
                count_num += curren_lemma.count()
        total_counts.append(count_num)
    
    max_count = max(total_counts)
    primary_sense = wordnet.synsets(word)[total_counts.index(max_count)]
    
    total_counts.sort( reverse = True )
    
    if total_counts[0] >= 4 * total_counts[1] and primary_sense.pos() == wordnet.NOUN:
        word_primary_sense[word] = primary_sense
        return True
    return False

for (key, value) in filtered_gold_standard.items():
    if filter_condition(key[0]) and filter_condition(key[1]):
        final_gold_standard[key] = value
###
# Your answer ENDS HERE
###
print(len(final_gold_standard))
print(final_gold_standard)

27
{('bread', 'butter'): 6.19, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('stock', 'egg'): 1.81, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('alcohol', 'chemistry'): 5.54, ('baby', 'mother'): 7.85, ('car', 'automobile'): 8.94, ('journey', 'voyage'): 9.29, ('coast', 'shore'): 9.1, ('furnace', 'stove'): 8.79, ('brother', 'monk'): 6.27, ('journey', 'car'): 5.85, ('coast', 'hill'): 4.38, ('forest', 'graveyard'): 1.85, ('monk', 'slave'): 0.92, ('coast', 'forest'): 3.15, ('psychology', 'doctor'): 6.42, ('psychology', 'mind'): 7.69, ('psychology', 'health'): 7.23, ('psychology', 'science'): 6.71, ('planet', 'moon'): 8.08, ('planet', 'galaxy'): 8.11}


<b>For your testing:</b>

In [151]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)

In [152]:
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Word similiarity scores with Lin similarity, NPMI and LSA (3 marks)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *lin_similarities*. 

(1 mark)

In [153]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###

brown_ic = wordnet_ic.ic('ic-brown.dat')

for key in final_gold_standard.keys():
    lin_similarities[key]=word_primary_sense[key[0]].lin_similarity(word_primary_sense[key[1]],brown_ic)
    
###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\Scott\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
{('bread', 'butter'): 0.711420490146294, ('professor', 'doctor'): 0.7036526610448273, ('student', 'professor'): 0.26208607023317687, ('stock', 'egg'): -0.0, ('money', 'cash'): 0.7888839126424345, ('king', 'queen'): 0.25872135992145145, ('bishop', 'rabbi'): 0.6655650900427844, ('football', 'basketball'): 0.7536025025710653, ('football', 'tennis'): 0.7699955045932811, ('alcohol', 'chemistry'): 0.062235427146896456, ('baby', 'mother'): 0.6315913189894092, ('car', 'automobile'): 1.0, ('journey', 'voyage'): 0.6969176573027711, ('coast', 'shore'): 0.9632173804623256, ('furnace', 'stove'): 0.22813808925013807, ('brother', 'monk'): 0.24862817480738675, ('journey', 'car'): -0.0, ('coast', 'hill'): 0.5991131628821826, ('forest', 'graveyard'): -0.0, ('monk', 'slave'): 0.2543108201944307, ('coast', 'forest'): -0.0, ('psychology', 'doctor'

<b>For your testing:</b>

In [154]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-\log_2(p(x,y))} = \frac{\log_2(p(x)p(y))}{\log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NMPI is -1. NPMI is normalized between [-1, +1].

You should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *NPMI_similarities*. 

(1 mark)

In [155]:
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###
import math


new_list=[]
for paragraphs in modified_paras:
    type_nums=set()
    for sentence in paragraphs:
        type_nums = type_nums.union(set(sentence))
    new_list.append(len(type_nums))
total_counts = sum(new_list)

def get_NPMI_for_collocation_brown(word1,word2):
    word1_counts, word2_counts = 0, 0
    both_counts = 0
    for paragraphs in modified_paras:
        word1_contained, word2_contained = False, False
        for sentence in paragraphs:
            if word1 in sentence:
                word1_contained = True
            if word2 in sentence:
                word2_contained = True
            
        if word1_contained:
            word1_counts += 1
        if word1_contained:
            word2_counts += 1
        if word1_contained and word2_contained:
            both_counts += 1
        
    
    if both_counts == 0:
        return -1
    else:
        PMI = math.log((both_counts * total_counts)/(word1_counts * word2_counts), 2)
        return PMI / ((-1) * math.log((both_counts/total_counts),2) )

for key in final_gold_standard.keys():
    NPMI_similarities[key] = get_NPMI_for_collocation_brown(key[0], key[1])




###
# Your answer ENDS HERE
###

print(NPMI_similarities)

689271 15667
689271
{('bread', 'butter'): 0.6066760645231779, ('professor', 'doctor'): -1, ('student', 'professor'): 0.4190718055298064, ('stock', 'egg'): 0.2793205940091102, ('money', 'cash'): 0.28540209894828594, ('king', 'queen'): 0.3444035598119812, ('bishop', 'rabbi'): -1, ('football', 'basketball'): 0.6115125977261736, ('football', 'tennis'): -1, ('alcohol', 'chemistry'): 0.6432603221169038, ('baby', 'mother'): 0.6070821537183281, ('car', 'automobile'): 0.4005240973398243, ('journey', 'voyage'): -1, ('coast', 'shore'): 0.5744960775040981, ('furnace', 'stove'): -1, ('brother', 'monk'): 0.3076210062091812, ('journey', 'car'): -1, ('coast', 'hill'): 0.3859979592090228, ('forest', 'graveyard'): -1, ('monk', 'slave'): -1, ('coast', 'forest'): 0.4613455903591904, ('psychology', 'doctor'): 0.618407351669097, ('psychology', 'mind'): 0.7063895592719672, ('psychology', 'health'): -1, ('psychology', 'science'): 0.76243604748902, ('planet', 'moon'): 0.6810753216595912, ('planet', 'galaxy'): 

<b>For your testing:</b>

In [95]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

**Instructions:** As PMI matrix is very sparse and can be approximated well by a dense representation via singular value decomposition (SVD), you will derive similarity scores using the Latent Semantic Analysis (LSA) method, i.e. apply SVD and truncate to get a dense vector representation of a word type and then calculate cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for your word pairs. 

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *LSA_similarities*. 

(1 mark)

In [114]:
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
def get_BOW(text):
    BOW = {}
    for paragraphs in text:
        for word in paragraphs:
            BOW[word] = BOW.get(word,0) + 1
    return BOW
texts = []
for fileid in modified_paras:
    texts.append(get_BOW(fileid))

vectorizer = DictVectorizer()
brown_matrix = vectorizer.fit_transform(texts)
brown_matrix_word_text=np.transpose(brown_matrix)
feature_names=vectorizer.get_feature_names()
#print(feature_names)
svd=TruncatedSVD(n_components=500)
brown_matrix_svd=svd.fit_transform(brown_matrix_word_text)

for key in final_gold_standard.keys():
    similarity=cosine_similarity(
        [brown_matrix_svd[feature_names.index(key[0])],brown_matrix_svd[feature_names.index(key[1])]])[0][1]
    LSA_similarities[key]=similarity
###
# Your answer ENDS HERE
###

print(LSA_similarities)

{('bread', 'butter'): 0.1342540457248209, ('professor', 'doctor'): 0.029087988442836277, ('student', 'professor'): 0.12039587883605751, ('stock', 'egg'): 0.026500647944975562, ('money', 'cash'): 0.16433512351349838, ('king', 'queen'): 0.07889731695513287, ('bishop', 'rabbi'): 0.015625329222944, ('football', 'basketball'): 0.216131101359676, ('football', 'tennis'): 0.10241529568595548, ('alcohol', 'chemistry'): 0.26241823148562227, ('baby', 'mother'): 0.21136571295632325, ('car', 'automobile'): 0.23956358320846483, ('journey', 'voyage'): 0.13024817391356486, ('coast', 'shore'): 0.2796667833077932, ('furnace', 'stove'): 0.18644337891480967, ('brother', 'monk'): 0.021645835425695627, ('journey', 'car'): -0.0027284915087364657, ('coast', 'hill'): 0.18875914357645335, ('forest', 'graveyard'): 0.011317485772835326, ('monk', 'slave'): -0.009915465884138587, ('coast', 'forest'): 0.039389420876220736, ('psychology', 'doctor'): 0.04230885967327654, ('psychology', 'mind'): 0.18319884679821266, ('

<b>For your testing:</b>

In [115]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## 3. Comparison with the Gold Standard (1 mark)


**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA'). You may check the section, <i>"For your testing"</i>, below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(1 mark)


In [163]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###
#print(final_gold_standard.values())
#print(tuple(final_gold_standard.values()))
#print(list(final_gold_standard.values()))
def pear(y_list):
    x_list, y_list =list(final_gold_standard.values()), list(y_list.values())

    return pearsonr(x_list,y_list)[0]


pearson_correlations['lin'] = pear(lin_similarities)
pearson_correlations['NPMI'] = pear(NPMI_similarities)
pearson_correlations['LSA'] = pear(LSA_similarities)
###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'lin': 0.44906965748915684, 'NPMI': 0.12301795067517061, 'LSA': 0.4650378347936122}


<b>For your testing:</b>

In [164]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## Challenge yourself: Improving the correlation (This part will NOT be marked)

You can try to derive a similarity score from word2vec vectors, using the Gensim interface, and compare it with the similarity scores you've created and the gold standard. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes).

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.